In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357288 sha256=92664b46560f4cfafdccdb1ba2ae2e56748de0ba44db5c4ed93ba0864365237d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

from surprise import Dataset, Reader
from surprise import KNNWithMeans, SVD, BaselineOnly, accuracy
from surprise.model_selection import train_test_split

import numpy as np

# Multinomical Classification

In [22]:
train_data = pd.read_csv('/content/drive/MyDrive/CS 229 Group Project/Data/Updated/train_data(non-zero).csv')
test_data = pd.read_csv('/content/drive/MyDrive/CS 229 Group Project/Data/Updated/test_data(non-zero).csv')

In [23]:
# get number of unique users and unique products in each dataset
num_users_train = train_data['user_id'].nunique()
num_products_train = train_data['parent_asin'].nunique()
print(f"Number of unique users in train data: {num_users_train}")
print(f"Number of unique products in train data: {num_products_train}")

num_users_test = test_data['user_id'].nunique()
num_products_test = test_data['parent_asin'].nunique()
print(f"Number of unique users in train data: {num_users_test}")
print(f"Number of unique products in train data: {num_products_test}")

Number of unique users in train data: 3245
Number of unique products in train data: 378
Number of unique users in train data: 4130
Number of unique products in train data: 661


In [24]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3431 entries, 0 to 3430
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   3431 non-null   object 
 1   parent_asin               3431 non-null   object 
 2   rating                    3431 non-null   float64
 3   user_average_rating       3431 non-null   float64
 4   user_std_rating           3431 non-null   float64
 5   avg_user_sentiment_title  3431 non-null   float64
 6   avg_user_sentiment_text   3431 non-null   float64
 7   user_review_count         3431 non-null   int64  
 8   helpfulness_ratio         3431 non-null   float64
 9   average_rating            3431 non-null   float64
 10  rating_number             3431 non-null   int64  
 11  title_word_count          3431 non-null   int64  
 12  weighted_rating           3431 non-null   float64
 13  description_sentiment     3431 non-null   float64
dtypes: float

In [25]:
user_features = [
    'user_average_rating', 'user_std_rating', 'avg_user_sentiment_title', 'avg_user_sentiment_text','user_review_count', 'helpfulness_ratio'
]

item_columns = ['average_rating', 'rating_number', 'title_word_count', 'weighted_rating', 'description_sentiment']

In [28]:
train_data['rating'] = train_data['rating'].astype(int)
test_data['rating'] = test_data['rating'].astype(int)

In [30]:
# Assuming X_test is a pandas DataFrame
nan_rows = test_data[test_data.isnull().any(axis=1)]

# Display the rows with NaN values
print(nan_rows)

# To get the indices of the rows with NaN values:
nan_indices = nan_rows.index
print(nan_indices)

# drop these
test_data = test_data.dropna()

                           user_id parent_asin  rating  user_average_rating  \
3445  AEAZU2CWMBHHJJA4VHMXBRRH5QOA  B08C2GDFYG       4             4.333333   
3446  AEBKQHXFWNNJSRJZ7DRARIOUAGRA  B08C2GDFYG       5             5.000000   
3447  AEFOKABSQH2FQGUTODBNSG44B6YQ  B08C2GDFYG       5             5.000000   

      user_std_rating  avg_user_sentiment_title  avg_user_sentiment_text  \
3445          1.21106                  0.246138                 0.185479   
3446          0.00000                  0.250000                 0.330556   
3447          0.00000                  0.425000                -0.012500   

      user_review_count  helpfulness_ratio  average_rating  rating_number  \
3445                  6                7.5             4.5            761   
3446                  2                0.0             4.5            761   
3447                  2                0.0             4.5            761   

      title_word_count  weighted_rating  description_sentiment  
3445

In [31]:
X_train = train_data[user_features + item_columns]
y_train = train_data['rating']
X_test = test_data[user_features + item_columns]
y_test = test_data['rating']

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Multinomial logistic regression
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced', multi_class='multinomial', solver='lbfgs')
log_reg.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = log_reg.predict(X_train_scaled)
y_test_pred = log_reg.predict(X_test_scaled)

# Metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))


Training Accuracy: 0.6674438939084815
Testing Accuracy: 0.6820137693631669
              precision    recall  f1-score   support

           1       0.41      0.94      0.58       413
           2       0.27      0.29      0.28       256
           3       0.49      0.26      0.34       404
           4       0.49      0.43      0.46       625
           5       0.87      0.79      0.83      2950

    accuracy                           0.68      4648
   macro avg       0.51      0.54      0.50      4648
weighted avg       0.71      0.68      0.68      4648



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


# Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = rf.predict(X_train_scaled)
y_test_pred = rf.predict(X_test_scaled)

# Metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

print(classification_report(y_test, y_test_pred))


Training Accuracy: 1.0
Testing Accuracy: 0.7312822719449226
              precision    recall  f1-score   support

           1       0.58      0.47      0.52       413
           2       0.57      0.18      0.28       256
           3       0.47      0.15      0.22       404
           4       0.61      0.39      0.47       625
           5       0.77      0.97      0.86      2950

    accuracy                           0.73      4648
   macro avg       0.60      0.43      0.47      4648
weighted avg       0.69      0.73      0.69      4648



# SVM

In [33]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', class_weight='balanced', probability=True)
svm.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = svm.predict(X_train_scaled)
y_test_pred = svm.predict(X_test_scaled)

# Metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

print(classification_report(y_test, y_test_pred))


Training Accuracy: 0.6499562809676479
Testing Accuracy: 0.6536144578313253
              precision    recall  f1-score   support

           1       0.47      0.86      0.61       413
           2       0.27      0.53      0.36       256
           3       0.37      0.49      0.42       404
           4       0.49      0.50      0.49       625
           5       0.93      0.69      0.79      2950

    accuracy                           0.65      4648
   macro avg       0.50      0.61      0.53      4648
weighted avg       0.74      0.65      0.68      4648



# XGBoost

In [34]:
from xgboost import XGBClassifier
xgb = XGBClassifier(objective='multi:softprob', num_class=5, eval_metric='mlogloss')
xgb.fit(X_train_scaled, y_train-1)

# Predictions
y_train_pred = xgb.predict(X_train_scaled)
y_test_pred = xgb.predict(X_test_scaled)

# Metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

print(classification_report(y_test, y_test_pred))

Training Accuracy: 0.0017487612940833576
Testing Accuracy: 0.07809810671256455
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.07      0.02      0.04       413
           2       0.14      0.13      0.14       256
           3       0.13      0.19      0.15       404
           4       0.07      0.39      0.12       625
           5       0.00      0.00      0.00      2950

    accuracy                           0.08      4648
   macro avg       0.07      0.12      0.08      4648
weighted avg       0.04      0.08      0.04      4648



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

# NN

In [35]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2)

test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

print(f"Test Accuracy: {test_accuracy:.2f}")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/pyt